In [1]:
# [Cell 1] - 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import roc_auc_score

In [2]:
# [Cell 2] - 데이터 로드 및 기본 정보 확인
df = pd.read_csv('./data/train.csv')
print("데이터 크기:", df.shape)
# print("\n기본 정보:")
# print(df.info())

데이터 크기: (256351, 69)


In [3]:
# [Cell 3] - 결측치 처리 및 기본 전처리
# 1. 95% 이상 결측치 컬럼 제거
missing_ratio = df.isnull().sum() / len(df) * 100
high_missing_cols = missing_ratio[missing_ratio > 95].index
df = df.drop(columns=high_missing_cols)

# 2. 범주형 변수 처리
categorical_cols = [
    '시술 시기 코드', '시술 당시 나이', '시술 유형', '특정 시술 유형',
    '배란 유도 유형', '난자 출처', '정자 출처'
]

# Label Encoding
le = LabelEncoder()
for col in categorical_cols:
    if col in df.columns:
        df[col] = le.fit_transform(df[col].astype(str))

print("처리 후 데이터 크기:", df.shape)
print("\n남은 결측치 비율:")
print(df.isnull().sum() / len(df) * 100)

처리 후 데이터 크기: (256351, 64)

남은 결측치 비율:
ID            0.000000
시술 시기 코드      0.000000
시술 당시 나이      0.000000
시술 유형         0.000000
특정 시술 유형      0.000000
               ...    
난자 채취 경과일    22.425503
난자 혼합 경과일    20.961494
배아 이식 경과일    16.994667
배아 해동 경과일    84.252451
임신 성공 여부      0.000000
Length: 64, dtype: float64


In [4]:
# 특정 시술 유형 컬럼의 데이터 타입과 unique 값들 확인
print("데이터 타입:", df['특정 시술 유형'].dtype)
print("\n고유값 샘플:")
print(df['특정 시술 유형'].unique()[:10])

데이터 타입: int64

고유값 샘플:
[ 4 15 23 14 20 18 12  5 11 19]


In [5]:
# 특정 시술 유형의 값별 분포와 각각의 임신 성공률을 확인
시술유형_성공률 = df.groupby('특정 시술 유형')['임신 성공 여부'].agg(['count', 'mean']).sort_values('mean', ascending=False)
print("시술 유형별 분포 및 성공률:")
print(시술유형_성공률)

시술 유형별 분포 및 성공률:
           count      mean
특정 시술 유형                  
8              1  1.000000
6              2  0.500000
18          1248  0.366186
7           1609  0.356743
0              3  0.333333
10             6  0.333333
19           392  0.283163
4         122368  0.272825
15         91755  0.261457
9              4  0.250000
23         26939  0.237203
12           873  0.234822
5            769  0.217165
16           319  0.184953
21           100  0.170000
2             71  0.169014
3             96  0.135417
14          6100  0.128525
13           207  0.101449
22            23  0.086957
20          1146  0.010471
11          2314  0.009939
1              2  0.000000
17             2  0.000000
24             2  0.000000


In [6]:
# [Cell 4] - 피처 엔지니어링

# 1. 시술 관련 복합 피처 생성
df['배아_성공_비율'] = df['이식된 배아 수'] / df['총 생성 배아 수'].replace(0, np.nan)
df['난자_성공_비율'] = df['미세주입된 난자 수'] / df['수집된 신선 난자 수'].replace(0, np.nan)

# 2. 시술 이력 관련 피처
시술횟수_컬럼 = ['총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수']
for col in 시술횟수_컬럼:
   # '6회 이상'을 6으로 변환
   df[col] = df[col].replace('6회 이상', '6')
   df[col] = df[col].str.replace('회', '').astype(float)

# 3. 불임 원인 복합 지표
불임원인_컬럼 = ['남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', 
           '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인']
df['총_불임원인_수'] = df[불임원인_컬럼].sum(axis=1)

# 4. 시술 유형 관련 피처
df['기본_시술'] = df['특정 시술 유형'].isin([4, 15]).astype(int)  # ICSI or IVF
df['BLASTOCYST_시술'] = df['특정 시술 유형'].isin([18, 7]).astype(int)  # with BLASTOCYST

print("생성된 피처 확인:")
new_features = ['배아_성공_비율', '난자_성공_비율', '총_불임원인_수', '기본_시술', 'BLASTOCYST_시술']
print(df[new_features].describe())

생성된 피처 확인:
            배아_성공_비율       난자_성공_비율       총_불임원인_수          기본_시술  \
count  196711.000000  196215.000000  256351.000000  256351.000000   
mean        0.367406       0.470087       0.126619       0.835273   
std         0.319077       0.419055       0.599379       0.370935   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.125000       0.000000       0.000000       1.000000   
50%         0.250000       0.583333       0.000000       1.000000   
75%         0.500000       0.875000       0.000000       1.000000   
max         3.000000       9.000000       6.000000       1.000000   

       BLASTOCYST_시술  
count  256351.000000  
mean        0.011145  
std         0.104980  
min         0.000000  
25%         0.000000  
50%         0.000000  
75%         0.000000  
max         1.000000  


In [7]:
# [Cell 4-1] - 배아/난자 성공 비율 이상치 처리
# 1. 이상치 확인
print("배아_성공_비율이 1 초과인 데이터 수:", (df['배아_성공_비율'] > 1).sum())
print("난자_성공_비율이 1 초과인 데이터 수:", (df['난자_성공_비율'] > 1).sum())

# 2. 이상치 처리 (1보다 큰 값은 1로 조정)
df['배아_성공_비율'] = df['배아_성공_비율'].clip(upper=1)
df['난자_성공_비율'] = df['난자_성공_비율'].clip(upper=1)

print("\n이상치 처리 후:")
print(df[['배아_성공_비율', '난자_성공_비율']].describe())

배아_성공_비율이 1 초과인 데이터 수: 181
난자_성공_비율이 1 초과인 데이터 수: 26

이상치 처리 후:
            배아_성공_비율       난자_성공_비율
count  196711.000000  196215.000000
mean        0.366442       0.469901
std         0.315422       0.418153
min         0.000000       0.000000
25%         0.125000       0.000000
50%         0.250000       0.583333
75%         0.500000       0.875000
max         1.000000       1.000000


In [8]:
# [Cell 4-2] - 불임 원인 그룹화 및 시술 관련 피처 최종 정리
# 1. 불임 원인 그룹화
df['불임원인_그룹'] = pd.cut(df['총_불임원인_수'], 
                        bins=[-np.inf, 0, 1, np.inf], 
                        labels=['없음', '단일', '복합'])

# 2. 최종 피처 구성
final_features = ['배아_성공_비율', '난자_성공_비율', '총_불임원인_수', 
                 '불임원인_그룹', '기본_시술', 'BLASTOCYST_시술']

print("최종 피처 확인:")
print(df[final_features].describe(include='all'))

최종 피처 확인:
             배아_성공_비율       난자_성공_비율       총_불임원인_수 불임원인_그룹          기본_시술  \
count   196711.000000  196215.000000  256351.000000  256351  256351.000000   
unique            NaN            NaN            NaN       3            NaN   
top               NaN            NaN            NaN      없음            NaN   
freq              NaN            NaN            NaN  245209            NaN   
mean         0.366442       0.469901       0.126619     NaN       0.835273   
std          0.315422       0.418153       0.599379     NaN       0.370935   
min          0.000000       0.000000       0.000000     NaN       0.000000   
25%          0.125000       0.000000       0.000000     NaN       1.000000   
50%          0.250000       0.583333       0.000000     NaN       1.000000   
75%          0.500000       0.875000       0.000000     NaN       1.000000   
max          1.000000       1.000000       6.000000     NaN       1.000000   

        BLASTOCYST_시술  
count   256351.000000  
uniqu

In [9]:
# [Cell 5] - 학습 데이터 준비

# 1. 타겟 변수 분리
X = df.drop(['임신 성공 여부', 'ID'], axis=1)
y = df['임신 성공 여부']

# 2. object 타입 컬럼 확인
object_columns = X.select_dtypes(include=['object']).columns
print("Object type columns:", object_columns.tolist())

# 3. 범주형 변수 전처리
categorical_features = ['불임원인_그룹', '시술 당시 나이', '시술 유형', '배란 유도 유형', 
                       '난자 출처', '정자 출처', '배아 생성 주요 이유', 
                       '클리닉 내 총 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 
                       'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 
                       'DI 출산 횟수', '난자 기증자 나이', '정자 기증자 나이']

X = pd.get_dummies(X, columns=categorical_features)

# 4. 결측치 처리
for col in X.select_dtypes(include=['float64', 'int64']).columns:
    X[col] = X[col].fillna(X[col].median())

# 5. 데이터 스케일링
scaler = StandardScaler()
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

print("학습 데이터 shape:", X.shape)
print("\n타겟 데이터 분포:")
print(y.value_counts(normalize=True))

Object type columns: ['배아 생성 주요 이유', '클리닉 내 총 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '난자 기증자 나이', '정자 기증자 나이']
학습 데이터 shape: (256351, 145)

타겟 데이터 분포:
임신 성공 여부
0    0.741651
1    0.258349
Name: proportion, dtype: float64


In [12]:
# [Cell 6] - Random Forest 모델 학습 및 교차 검증
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

# 1. 모델 설정
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)

# 2. 교차 검증 수행
cv_scores = cross_val_score(
    rf_model, 
    X, 
    y, 
    cv=5, 
    scoring='roc_auc',
    n_jobs=-1
)

print(f'교차 검증 ROC-AUC 점수: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})')

# 3. 전체 데이터로 학습 및 피처 중요도 확인
rf_model.fit(X, y)

# 피처 중요도 상위 20개 출력
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
})
feature_importance = feature_importance.sort_values('importance', ascending=False)

print('\n상위 20개 중요 특성:')
print(feature_importance.head(20))

교차 검증 ROC-AUC 점수: 0.7296 (+/- 0.0015)

상위 20개 중요 특성:
               feature  importance
28            이식된 배아 수    0.195476
45           배아 이식 경과일    0.138609
47            배아_성공_비율    0.119037
25           총 생성 배아 수    0.056438
30            저장된 배아 수    0.054767
55          시술 당시 나이_0    0.033679
36            혼합된 난자 수    0.033368
37    파트너 정자와 혼합된 난자 수    0.033060
3          단일 배아 이식 여부    0.030658
29        미세주입 배아 이식 수    0.027398
34         수집된 신선 난자 수    0.023053
31     미세주입 후 저장된 배아 수    0.022871
84  배아 생성 주요 이유_배아 저장용    0.022211
58          시술 당시 나이_3    0.021283
27     미세주입에서 생성된 배아 수    0.020253
87  배아 생성 주요 이유_현재 시술용    0.016383
59          시술 당시 나이_4    0.012167
1             특정 시술 유형    0.011559
26          미세주입된 난자 수    0.009748
56          시술 당시 나이_1    0.005400


In [ ]:
# [Cell 7] - Random Forest 하이퍼파라미터 최적화
# 너무 오래걸려서 아래와 같이 나눠서 변경
from sklearn.model_selection import GridSearchCV

# 1. 탐색할 하이퍼파라미터 정의
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# 2. GridSearchCV 설정
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1)
grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=2
)

# 3. 그리드 서치 수행
grid_search.fit(X, y)

# 4. 최적 파라미터와 점수 출력
print("최적 파라미터:", grid_search.best_params_)
print(f"최고 ROC-AUC 점수: {grid_search.best_score_:.4f}")

In [13]:
# [Cell 7-1] - 첫 단계 최적화 (n_estimators, max_depth)
from sklearn.model_selection import GridSearchCV

# 1. 주요 파라미터만 먼저 탐색
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 15, 20]
}

rf_model = RandomForestClassifier(
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1
)

grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X, y)

print("최적 파라미터:", grid_search.best_params_)
print(f"최고 ROC-AUC 점수: {grid_search.best_score_:.4f}")

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] END .....................max_depth=10, n_estimators=100; total time=  29.3s
[CV] END .....................max_depth=10, n_estimators=100; total time=  29.4s
[CV] END .....................max_depth=10, n_estimators=100; total time=  29.6s
[CV] END .....................max_depth=10, n_estimators=100; total time=  30.4s
[CV] END .....................max_depth=10, n_estimators=100; total time=  33.4s
[CV] END .....................max_depth=10, n_estimators=200; total time=  53.1s
[CV] END .....................max_depth=10, n_estimators=200; total time=  56.0s
[CV] END .....................max_depth=15, n_estimators=100; total time=  43.1s
[CV] END .....................max_depth=15, n_estimators=100; total time=  43.0s
[CV] END .....................max_depth=10, n_estimators=200; total time=  55.2s
[CV] END .....................max_depth=10, n_estimators=200; total time=  57.6s
[CV] END .....................max_depth=10, n_est

In [14]:
# [Cell 7-2] - 두 번째 단계 최적화 (min_samples_split, min_samples_leaf)
param_grid = {
    'min_samples_split': [2, 5, 8],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    random_state=42,
    n_jobs=-1
)

grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X, y)

print("최적 파라미터:", grid_search.best_params_)
print(f"최고 ROC-AUC 점수: {grid_search.best_score_:.4f}")

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END max_features=sqrt, min_samples_leaf=1, min_samples_split=5; total time= 1.3min
[CV] END max_features=sqrt, min_samples_leaf=1, min_samples_split=2; total time= 1.3min
[CV] END max_features=sqrt, min_samples_leaf=1, min_samples_split=2; total time= 1.3min
[CV] END max_features=sqrt, min_samples_leaf=1, min_samples_split=2; total time= 1.4min
[CV] END max_features=sqrt, min_samples_leaf=1, min_samples_split=5; total time= 1.4min
[CV] END max_features=sqrt, min_samples_leaf=1, min_samples_split=2; total time= 1.4min
[CV] END max_features=sqrt, min_samples_leaf=1, min_samples_split=2; total time= 1.4min
[CV] END max_features=sqrt, min_samples_leaf=1, min_samples_split=5; total time= 1.3min
[CV] END max_features=sqrt, min_samples_leaf=1, min_samples_split=8; total time= 1.3min
[CV] END max_features=sqrt, min_samples_leaf=1, min_samples_split=5; total time= 1.4min
[CV] END max_features=sqrt, min_samples_leaf=1, min_samples

/data/ephemeral/home/.cache/pypoetry/virtualenvs/fertimind-C3Sc56E3-py3.11/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END max_features=log2, min_samples_leaf=2, min_samples_split=5; total time=  54.0s
[CV] END max_features=log2, min_samples_leaf=2, min_samples_split=5; total time=  54.3s
[CV] END max_features=log2, min_samples_leaf=2, min_samples_split=5; total time=  56.6s
[CV] END max_features=log2, min_samples_leaf=2, min_samples_split=8; total time=  50.7s
[CV] END max_features=log2, min_samples_leaf=2, min_samples_split=8; total time=  48.7s
[CV] END max_features=log2, min_samples_leaf=2, min_samples_split=8; total time=  54.6s
[CV] END max_features=log2, min_samples_leaf=2, min_samples_split=8; total time=  54.1s
[CV] END max_features=log2, min_samples_leaf=4, min_samples_split=2; total time=  54.6s
[CV] END max_features=log2, min_samples_leaf=4, min_samples_split=2; total time=  54.7s
[CV] END max_features=log2, min_samples_leaf=2, min_samples_split=8; total time=  55.9s
[CV] END max_features=log2, min_samples_leaf=4, min_samples_split=2; total time=  53.9s
[CV] END max_features=log2, min_

In [17]:
# [Cell 8] - XGBoost 기본 모델
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold

# 기본 모델 정의
xgb_model = XGBClassifier(
    random_state=42,
    n_jobs=-1
)

# StratifiedKFold 정의
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 수행
cv_scores = []

for train_idx, valid_idx in skf.split(X, y):
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
    
    # 모델 학습
    xgb_model.fit(X_train, y_train)
    
    # 예측 및 ROC-AUC 계산
    y_pred = xgb_model.predict_proba(X_valid)[:, 1]
    score = roc_auc_score(y_valid, y_pred)
    cv_scores.append(score)

print(f'XGBoost 기본 모델 교차 검증 ROC-AUC 점수: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})')

XGBoost 기본 모델 교차 검증 ROC-AUC 점수: 0.7356 (+/- 0.0020)


In [20]:
# [Cell 8-1] - XGBoost 첫 단계 최적화 (n_estimators, max_depth, learning_rate)
param_grid = {
   'n_estimators': [100, 200],
   'max_depth': [3, 6, 9],
   'learning_rate': [0.01, 0.1]
}

# Grid Search와 교차 검증
best_score = 0
best_params = None
scores = []

# StratifiedKFold 정의
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 모든 파라미터 조합에 대해 반복
for n_est in param_grid['n_estimators']:
   for depth in param_grid['max_depth']:
       for lr in param_grid['learning_rate']:
           cv_scores = []
           
           # 각 fold에 대해 학습 및 평가
           for train_idx, valid_idx in skf.split(X, y):
               X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
               y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
               
               # 모델 정의 및 학습
               model = XGBClassifier(
                   n_estimators=n_est,
                   max_depth=depth,
                   learning_rate=lr,
                   random_state=42,
                   n_jobs=-1
               )
               
               model.fit(X_train, y_train)
               
               # 예측 및 점수 계산
               y_pred = model.predict_proba(X_valid)[:, 1]
               score = roc_auc_score(y_valid, y_pred)
               cv_scores.append(score)
           
           # 평균 점수 계산
           mean_score = np.mean(cv_scores)
           scores.append({
               'n_estimators': n_est,
               'max_depth': depth,
               'learning_rate': lr,
               'score': mean_score
           })
           
           # 최고 점수 갱신
           if mean_score > best_score:
               best_score = mean_score
               best_params = {
                   'n_estimators': n_est,
                   'max_depth': depth,
                   'learning_rate': lr
               }

print("최적 파라미터:", best_params)
print(f"최고 ROC-AUC 점수: {best_score:.4f}")

# 모든 결과를 데이터프레임으로 변환하여 확인
results_df = pd.DataFrame(scores)
print("\n모든 파라미터 조합 결과:")
print(results_df.sort_values('score', ascending=False).head())

최적 파라미터: {'n_estimators': 100, 'max_depth': 6, 'learning_rate': 0.1}
최고 ROC-AUC 점수: 0.7387

모든 파라미터 조합 결과:
   n_estimators  max_depth  learning_rate     score
3           100          6            0.1  0.738729
7           200          3            0.1  0.738547
9           200          6            0.1  0.738116
5           100          9            0.1  0.736699
1           100          3            0.1  0.736552


In [21]:
# [Cell 8-2] - XGBoost 두 번째 단계 최적화 (min_child_weight, subsample, colsample_bytree)
param_grid = {
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# 이전 단계에서 찾은 최적 파라미터 적용
best_params_prev = best_params  # 8-1에서 찾은 최적 파라미터

# Grid Search와 교차 검증
best_score = 0
best_params = None
scores = []

# StratifiedKFold 정의
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 모든 파라미터 조합에 대해 반복
for mcw in param_grid['min_child_weight']:
    for ss in param_grid['subsample']:
        for cs in param_grid['colsample_bytree']:
            cv_scores = []
            
            # 각 fold에 대해 학습 및 평가
            for train_idx, valid_idx in skf.split(X, y):
                X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
                y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
                
                # 모델 정의 및 학습 (이전 최적 파라미터 포함)
                model = XGBClassifier(
                    n_estimators=best_params_prev['n_estimators'],
                    max_depth=best_params_prev['max_depth'],
                    learning_rate=best_params_prev['learning_rate'],
                    min_child_weight=mcw,
                    subsample=ss,
                    colsample_bytree=cs,
                    random_state=42,
                    n_jobs=-1
                )
                
                model.fit(X_train, y_train)
                
                # 예측 및 점수 계산
                y_pred = model.predict_proba(X_valid)[:, 1]
                score = roc_auc_score(y_valid, y_pred)
                cv_scores.append(score)
            
            # 평균 점수 계산
            mean_score = np.mean(cv_scores)
            scores.append({
                'min_child_weight': mcw,
                'subsample': ss,
                'colsample_bytree': cs,
                'score': mean_score
            })
            
            # 최고 점수 갱신
            if mean_score > best_score:
                best_score = mean_score
                best_params = {
                    'min_child_weight': mcw,
                    'subsample': ss,
                    'colsample_bytree': cs
                }

print("최적 파라미터:", best_params)
print(f"최고 ROC-AUC 점수: {best_score:.4f}")

# 모든 결과를 데이터프레임으로 변환하여 확인
results_df = pd.DataFrame(scores)
print("\n모든 파라미터 조합 결과:")
print(results_df.sort_values('score', ascending=False).head())

최적 파라미터: {'min_child_weight': 5, 'subsample': 0.8, 'colsample_bytree': 0.9}
최고 ROC-AUC 점수: 0.7391

모든 파라미터 조합 결과:
    min_child_weight  subsample  colsample_bytree     score
19                 5        0.8               0.9  0.739144
1                  1        0.8               0.9  0.739122
22                 5        0.9               0.9  0.739065
0                  1        0.8               0.8  0.739057
20                 5        0.8               1.0  0.739037


In [10]:
from catboost import CatBoostClassifier

# [Cell 9] - CatBoost 기본 모델
cat_model = CatBoostClassifier(
    random_state=42,
    verbose=False
)

# 교차 검증 수행
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

for train_idx, valid_idx in skf.split(X, y):
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
    
    # 모델 학습
    cat_model.fit(X_train, y_train)
    
    # 예측 및 ROC-AUC 계산
    y_pred = cat_model.predict_proba(X_valid)[:, 1]
    score = roc_auc_score(y_valid, y_pred)
    cv_scores.append(score)

print(f'CatBoost 기본 모델 교차 검증 ROC-AUC 점수: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})')

CatBoost 기본 모델 교차 검증 ROC-AUC 점수: 0.7371 (+/- 0.0024)


In [11]:
# [Cell 9-1] - CatBoost 첫 단계 최적화 (iterations, learning_rate, depth)
param_grid = {
    'iterations': [100, 200],
    'learning_rate': [0.01, 0.1],
    'depth': [4, 6, 8]
}

# Grid Search와 교차 검증
best_score = 0
best_params = None
scores = []

# StratifiedKFold 정의
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 모든 파라미터 조합에 대해 반복
for iter_ in param_grid['iterations']:
    for lr in param_grid['learning_rate']:
        for depth in param_grid['depth']:
            cv_scores = []
            
            # 각 fold에 대해 학습 및 평가
            for train_idx, valid_idx in skf.split(X, y):
                X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
                y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
                
                # 모델 정의 및 학습
                model = CatBoostClassifier(
                    iterations=iter_,
                    learning_rate=lr,
                    depth=depth,
                    random_state=42,
                    verbose=False
                )
                
                model.fit(X_train, y_train)
                
                # 예측 및 점수 계산
                y_pred = model.predict_proba(X_valid)[:, 1]
                score = roc_auc_score(y_valid, y_pred)
                cv_scores.append(score)
            
            # 평균 점수 계산
            mean_score = np.mean(cv_scores)
            scores.append({
                'iterations': iter_,
                'learning_rate': lr,
                'depth': depth,
                'score': mean_score
            })
            
            # 최고 점수 갱신
            if mean_score > best_score:
                best_score = mean_score
                best_params = {
                    'iterations': iter_,
                    'learning_rate': lr,
                    'depth': depth
                }

print("최적 파라미터:", best_params)
print(f"최고 ROC-AUC 점수: {best_score:.4f}")

# 모든 결과를 데이터프레임으로 변환하여 확인
results_df = pd.DataFrame(scores)
print("\n모든 파라미터 조합 결과:")
print(results_df.sort_values('score', ascending=False).head())

최적 파라미터: {'iterations': 200, 'learning_rate': 0.1, 'depth': 6}
최고 ROC-AUC 점수: 0.7392

모든 파라미터 조합 결과:
    iterations  learning_rate  depth     score
10         200            0.1      6  0.739192
9          200            0.1      4  0.738937
11         200            0.1      8  0.738783
5          100            0.1      8  0.738678
4          100            0.1      6  0.738512


In [12]:
# [Cell 9-2] - CatBoost 두 번째 단계 최적화 (l2_leaf_reg, rsm, subsample)
param_grid = {
   'l2_leaf_reg': [1, 3, 5],        # L2 regularization
   'rsm': [0.8, 0.9, 1.0],          # random subspace method (colsample_bylevel)
   'subsample': [0.8, 0.9, 1.0]     # bagging 관련 파라미터
}

# 이전 단계에서 찾은 최적 파라미터 적용
best_params_prev = best_params  # 9-1에서 찾은 최적 파라미터

# Grid Search와 교차 검증
best_score = 0
best_params = None
scores = []

# StratifiedKFold 정의
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 모든 파라미터 조합에 대해 반복
for l2 in param_grid['l2_leaf_reg']:
   for rsm in param_grid['rsm']:
       for ss in param_grid['subsample']:
           cv_scores = []
           
           # 각 fold에 대해 학습 및 평가
           for train_idx, valid_idx in skf.split(X, y):
               X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
               y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
               
               # 모델 정의 및 학습 (이전 최적 파라미터 포함)
               model = CatBoostClassifier(
                   iterations=best_params_prev['iterations'],
                   learning_rate=best_params_prev['learning_rate'],
                   depth=best_params_prev['depth'],
                   l2_leaf_reg=l2,
                   rsm=rsm,
                   subsample=ss,
                   random_state=42,
                   verbose=False
               )
               
               model.fit(X_train, y_train)
               
               # 예측 및 점수 계산
               y_pred = model.predict_proba(X_valid)[:, 1]
               score = roc_auc_score(y_valid, y_pred)
               cv_scores.append(score)
           
           # 평균 점수 계산
           mean_score = np.mean(cv_scores)
           scores.append({
               'l2_leaf_reg': l2,
               'rsm': rsm,
               'subsample': ss,
               'score': mean_score
           })
           
           # 최고 점수 갱신
           if mean_score > best_score:
               best_score = mean_score
               best_params = {
                   'l2_leaf_reg': l2,
                   'rsm': rsm,
                   'subsample': ss
               }

print("최적 파라미터:", best_params)
print(f"최고 ROC-AUC 점수: {best_score:.4f}")

# 모든 결과를 데이터프레임으로 변환하여 확인
results_df = pd.DataFrame(scores)
print("\n모든 파라미터 조합 결과:")
print(results_df.sort_values('score', ascending=False).head())

최적 파라미터: {'l2_leaf_reg': 5, 'rsm': 0.8, 'subsample': 0.8}
최고 ROC-AUC 점수: 0.7394

모든 파라미터 조합 결과:
    l2_leaf_reg  rsm  subsample     score
18            5  0.8        0.8  0.739418
20            5  0.8        1.0  0.739415
21            5  0.9        0.8  0.739407
25            5  1.0        0.9  0.739360
23            5  0.9        1.0  0.739340


In [16]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold

In [17]:
# [Cell 10] - 최종 앙상블 모델 구성
# 1. 최적화된 각 모델 정의
rf_final = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1
)

xgb_final = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    min_child_weight=5,
    subsample=0.8,
    colsample_bytree=0.9,
    random_state=42,
    n_jobs=-1
)

cat_final = CatBoostClassifier(
    iterations=200,
    learning_rate=0.1,
    depth=6,
    l2_leaf_reg=5,
    rsm=0.8,
    subsample=0.8,
    random_state=42,
    verbose=False
)

# 2. 각 모델의 교차 검증 예측값 생성
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
predictions = np.zeros((len(X), 3))  # 각 모델의 예측값을 저장할 배열

for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
    
    # 각 모델 학습 및 예측
    rf_final.fit(X_train, y_train)
    xgb_final.fit(X_train, y_train)
    cat_final.fit(X_train, y_train)
    
    predictions[valid_idx, 0] = rf_final.predict_proba(X_valid)[:, 1]
    predictions[valid_idx, 1] = xgb_final.predict_proba(X_valid)[:, 1]
    predictions[valid_idx, 2] = cat_final.predict_proba(X_valid)[:, 1]

# 3. 단순 평균 앙상블
ensemble_pred = predictions.mean(axis=1)
ensemble_score = roc_auc_score(y, ensemble_pred)

print(f'앙상블 모델 ROC-AUC 점수: {ensemble_score:.4f}')

# 4. 각 모델별 성능 비교
print("\n개별 모델 성능:")
print(f'Random Forest ROC-AUC: {roc_auc_score(y, predictions[:, 0]):.4f}')
print(f'XGBoost ROC-AUC: {roc_auc_score(y, predictions[:, 1]):.4f}')
print(f'CatBoost ROC-AUC: {roc_auc_score(y, predictions[:, 2]):.4f}')

앙상블 모델 ROC-AUC 점수: 0.7391

개별 모델 성능:
Random Forest ROC-AUC: 0.7340
XGBoost ROC-AUC: 0.7391
CatBoost ROC-AUC: 0.7394


In [19]:
# [Cell 11-1] - 수정된 가중치 앙상블 모델

# 1. 성능 차이를 더 강조한 가중치 계산
# 기본 점수(0.73)를 뺀 후의 차이를 기반으로 가중치 계산
base_score = 0.73
rf_performance = 0.7340 - base_score
xgb_performance = 0.7391 - base_score
cat_performance = 0.7394 - base_score

# 성능 차이를 제곱하여 더 강조
rf_weight = rf_performance**2
xgb_weight = xgb_performance**2
cat_weight = cat_performance**2

# 가중치 정규화
total_weight = rf_weight + xgb_weight + cat_weight
rf_weight = rf_weight / total_weight
xgb_weight = xgb_weight / total_weight
cat_weight = cat_weight / total_weight

weights = np.array([rf_weight, xgb_weight, cat_weight])
print("수정된 가중치:")
print(f'Random Forest: {rf_weight:.4f}')
print(f'XGBoost: {xgb_weight:.4f}')
print(f'CatBoost: {cat_weight:.4f}')

# 2. 새로운 가중치 적용 앙상블
weighted_pred = np.average(predictions, weights=weights, axis=1)
weighted_score = roc_auc_score(y, weighted_pred)

print(f'\n가중치 앙상블 모델 ROC-AUC 점수: {weighted_score:.4f}')

수정된 가중치:
Random Forest: 0.0855
XGBoost: 0.4424
CatBoost: 0.4721

가중치 앙상블 모델 ROC-AUC 점수: 0.7396


### 제출결과 0.67-

In [25]:
# [Cell 12] - 테스트 데이터 예측
# 1. 테스트 데이터 로드 및 전처리
test_df = pd.read_csv('./data/test.csv')

# 2. 전처리 과정 적용 (train과 동일하게)

# 범주형 변수 처리 (Cell 3과 동일하게)
categorical_cols = [
    '시술 시기 코드', '시술 당시 나이', '시술 유형', '특정 시술 유형',
    '배란 유도 유형', '난자 출처', '정자 출처'
]

# Label Encoding
le = LabelEncoder()
for col in categorical_cols:
    if col in test_df.columns:
        test_df[col] = le.fit_transform(test_df[col].astype(str))

# Feature Engineering
# 배아 관련 피처
test_df['배아_성공_비율'] = test_df['이식된 배아 수'] / test_df['총 생성 배아 수'].replace(0, np.nan)
test_df['난자_성공_비율'] = test_df['미세주입된 난자 수'] / test_df['수집된 신선 난자 수'].replace(0, np.nan)

# 시술 횟수 변환
시술횟수_컬럼 = ['총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수']
for col in 시술횟수_컬럼:
    test_df[col] = test_df[col].replace('6회 이상', '6')
    test_df[col] = test_df[col].str.replace('회', '').astype(float)

# 불임 원인 복합 지표
불임원인_컬럼 = ['남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', 
            '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인']
test_df['총_불임원인_수'] = test_df[불임원인_컬럼].sum(axis=1)
test_df['불임원인_그룹'] = pd.cut(test_df['총_불임원인_수'], 
                        bins=[-np.inf, 0, 1, np.inf], 
                        labels=['없음', '단일', '복합'])

# 시술 유형 관련 피처
test_df['기본_시술'] = test_df['특정 시술 유형'].isin([4, 15]).astype(int)
test_df['BLASTOCYST_시술'] = test_df['특정 시술 유형'].isin([18, 7]).astype(int)

# 3. 학습 데이터와 동일한 전처리 적용
categorical_features = ['불임원인_그룹', '시술 당시 나이', '시술 유형', '배란 유도 유형', 
                       '난자 출처', '정자 출처', '배아 생성 주요 이유', 
                       '클리닉 내 총 시술 횟수', '총 임신 횟수', 'IVF 임신 횟수', 
                       'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 
                       'DI 출산 횟수', '난자 기증자 나이', '정자 기증자 나이']

X_test = pd.get_dummies(test_df.drop(['ID'], axis=1), columns=categorical_features)

# 학습 데이터와 컬럼 일치시키기
for col in X.columns:
    if col not in X_test.columns:
        X_test[col] = 0
X_test = X_test[X.columns]

# 4. 예측
rf_pred = rf_final.predict_proba(X_test)[:, 1]
xgb_pred = xgb_final.predict_proba(X_test)[:, 1]
cat_pred = cat_final.predict_proba(X_test)[:, 1]

# 가중치 적용한 최종 예측
final_pred = (rf_pred * rf_weight + 
             xgb_pred * xgb_weight + 
             cat_pred * cat_weight)

# 5. 제출 파일 생성
submission = pd.DataFrame({
    'ID': test_df['ID'],
    'probability': final_pred
})
submission.to_csv('./data/submission.csv', index=False)

print("예측 완료 및 제출 파일 생성")
print("\n예측값 통계:")
print(submission['probability'].describe())

예측 완료 및 제출 파일 생성

예측값 통계:
count    90067.000000
mean         0.300300
std          0.106085
min          0.048214
25%          0.226929
50%          0.293111
75%          0.367998
max          0.821589
Name: probability, dtype: float64


--------

In [26]:
# [Cell 13] - 시계열 특성 파악
# 1. 시술 시기 코드별 성공률 확인
time_success = df.groupby('시술 시기 코드')['임신 성공 여부'].agg(['count', 'mean'])
time_success.columns = ['시술 건수', '성공률']
print("시기별 성공률:")
print(time_success)

# 2. 시기별 주요 변수들의 분포 변화
key_features = ['이식된 배아 수', '총 생성 배아 수', 'BLASTOCYST_시술']
for feature in key_features:
    print(f"\n{feature}의 시기별 평균:")
    print(df.groupby('시술 시기 코드')[feature].mean())

시기별 성공률:
          시술 건수       성공률
시술 시기 코드                 
0         38090  0.257417
1         38969  0.245426
2         36031  0.266271
3         36173  0.259779
4         34831  0.256122
5         36713  0.269087
6         35544  0.255120

이식된 배아 수의 시기별 평균:
시술 시기 코드
0    1.225670
1    1.154085
2    1.394465
3    1.451753
4    1.548104
5    1.327421
6    1.510573
Name: 이식된 배아 수, dtype: float64

총 생성 배아 수의 시기별 평균:
시술 시기 코드
0    5.035885
1    4.993185
2    5.147988
3    5.036626
4    5.029231
5    5.094266
6    5.096568
Name: 총 생성 배아 수, dtype: float64

BLASTOCYST_시술의 시기별 평균:
시술 시기 코드
0    0.000919
1    0.000334
2    0.002165
3    0.007022
4    0.033763
5    0.001553
6    0.034999
Name: BLASTOCYST_시술, dtype: float64


In [27]:
# [Cell 14] - 테스트 데이터 시기별 분석
# 1. 테스트 데이터 시기 분포
time_dist_test = test_df['시술 시기 코드'].value_counts().sort_index()
print("테스트 데이터 시기별 분포:")
print(time_dist_test)

# 2. 테스트 데이터 시기별 주요 특성
key_features = ['이식된 배아 수', '총 생성 배아 수', 'BLASTOCYST_시술']
for feature in key_features:
    print(f"\n{feature}의 시기별 평균 (테스트 데이터):")
    print(test_df.groupby('시술 시기 코드')[feature].mean())

테스트 데이터 시기별 분포:
시술 시기 코드
0    13392
1    13610
2    12764
3    12560
4    12019
5    12929
6    12793
Name: count, dtype: int64

이식된 배아 수의 시기별 평균 (테스트 데이터):
시술 시기 코드
0    1.227724
1    1.146094
2    1.394066
3    1.460372
4    1.559354
5    1.337637
6    1.500360
Name: 이식된 배아 수, dtype: float64

총 생성 배아 수의 시기별 평균 (테스트 데이터):
시술 시기 코드
0    5.042112
1    4.936820
2    5.112969
3    5.005000
4    5.087428
5    5.090052
6    5.154111
Name: 총 생성 배아 수, dtype: float64

BLASTOCYST_시술의 시기별 평균 (테스트 데이터):
시술 시기 코드
0    0.003062
1    0.003747
2    0.002664
3    0.002787
4    0.003494
5    0.003945
6    0.004065
Name: BLASTOCYST_시술, dtype: float64


In [28]:
# [Cell 15] - 예측값 상세 분석
# submission 파일 다시 읽기
submission = pd.read_csv('./data/submission.csv')

# test 데이터와 예측값 합치기
test_with_pred = test_df.copy()
test_with_pred['predicted_prob'] = submission['probability']

# 1. BLASTOCYST 시술 여부에 따른 예측값 분포
print("BLASTOCYST 시술 여부에 따른 예측값 통계:")
print(test_with_pred.groupby('BLASTOCYST_시술')['predicted_prob'].describe())

# 2. 시기별 예측값 분포
print("\n시기별 예측값 통계:")
print(test_with_pred.groupby('시술 시기 코드')['predicted_prob'].describe())

BLASTOCYST 시술 여부에 따른 예측값 통계:
                 count      mean       std       min       25%       50%  \
BLASTOCYST_시술                                                              
0              89761.0  0.300333  0.106072  0.048214  0.226985  0.293142   
1                306.0  0.290798  0.109582  0.072956  0.218810  0.278349   

                    75%       max  
BLASTOCYST_시술                      
0              0.368007  0.821589  
1              0.349055  0.706720  

시기별 예측값 통계:
            count      mean       std       min       25%       50%       75%  \
시술 시기 코드                                                                        
0         13392.0  0.311451  0.113988  0.048214  0.229782  0.303960  0.385940   
1         13610.0  0.300643  0.111089  0.054490  0.220654  0.295047  0.373923   
2         12764.0  0.298435  0.103021  0.050654  0.227438  0.291717  0.363839   
3         12560.0  0.298526  0.102606  0.052335  0.228292  0.291735  0.362998   
4         12019.0  0.29

In [29]:
# [Cell 16] - BLASTOCYST 시술과 다른 변수들의 관계 분석 (학습 데이터)
# 1. BLASTOCYST 시술과 주요 변수들의 관계
key_features = ['이식된 배아 수', '총 생성 배아 수', '배아_성공_비율', '임신 성공 여부']

print("BLASTOCYST 시술에 따른 주요 변수 통계 (학습 데이터):")
for feature in key_features:
   print(f"\n{feature}:")
   print(df.groupby('BLASTOCYST_시술')[feature].describe())

# 2. BLASTOCYST 시술과 시기별 성공률
print("\n시기별 BLASTOCYST 시술 성공률:")
print(df.groupby(['시술 시기 코드', 'BLASTOCYST_시술'])['임신 성공 여부'].mean().unstack())

BLASTOCYST 시술에 따른 주요 변수 통계 (학습 데이터):

이식된 배아 수:
                  count      mean       std  min  25%  50%  75%  max
BLASTOCYST_시술                                                       
0              247203.0  1.367002  0.773873  0.0  1.0  1.0  2.0  3.0
1                2857.0  1.467973  0.574712  0.0  1.0  1.0  2.0  3.0

총 생성 배아 수:
                  count      mean       std  min  25%  50%   75%   max
BLASTOCYST_시술                                                         
0              247203.0  5.039174  4.655951  0.0  1.0  4.0   8.0  51.0
1                2857.0  6.962198  4.987166  0.0  4.0  7.0  10.0  34.0

배아_성공_비율:
                  count      mean       std  min       25%       50%   75%  \
BLASTOCYST_시술                                                                
0              194495.0  0.368362  0.316439  0.0  0.125000  0.250000  0.50   
1                2216.0  0.197952  0.119828  0.0  0.111111  0.166667  0.25   

               max  
BLASTOCYST_시술       
0             